In [1]:
!pip install minio
import minio
from minio import Minio

### Bucket mlpipeline 조회 

In [2]:
minioClient = Minio('minio-service.kubeflow.svc.cluster.local:9000',
                     access_key='minio',
                     secret_key='minio123',
                     secure=False)

bucket_name = "mlpipeline"

if minioClient.bucket_exists(bucket_name):
    print(f"{bucket_name} exists")
else:
    print(f"{bucket_name} does not exist")

mlpipeline exists


### 공통 클래스 생성 
- create_bucket
- upload_file
- upload_directory

In [4]:
class MinioClient():
    def __init__(self):
        self.client = Minio(
            "minio-service.kubeflow:9000",
            access_key="minio",
            secret_key="minio123",
            secure=False
        )

    def create_bucket(self,bucket_name):
        if self.client.bucket_exists(bucket_name):
            print(f"{bucket_name} exists")
        else:
            self.client.make_bucket(bucket_name)        
            print(f"{bucket_name} created")

    def upload_file(self, bucket, minio_path, local_path):
        self.client.fput_object(bucket, minio_path, local_path)

    def upload_directory(self, bucket, minio_path, local_path):
        if os.path.isdir(local_path):
            items = os.listdir(local_path)
            items.sort()
            for item in items:
                item_local_path = os.path.join(local_path, item)
                item_minio_path = os.path.join(minio_path, item)
                if os.path.isfile(item_local_path):
                    self.upload_file(bucket, item_minio_path, item_local_path)
                elif os.path.isdir(item_local_path):
                    self.upload_directory(bucket, item_minio_path, item_local_path)

### bucket 생성

In [5]:
mc_sdk = MinioClient()

In [6]:
mc_sdk.create_bucket("test-sdk")

test-sdk created


### upload file

In [7]:
mc_sdk.upload_file("test-sdk", "test.txt", "test.txt")